In [188]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import sys
from IPython.display import display

In [3]:
# impot hostel and user data for finding distance 
hostel_dist = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/hostel_dist_oct10.csv')
user_dist = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/user_dist_oct10.csv')

In [262]:
user_dist.head()

,Unnamed: 0,user_id,user_stuff,hsts_liked
0,0,1189474,"[Fortuna Botel, Hungary, 7-8.5, Hs:1, Rt6, nat...",[Fortuna Botel]
1,1,1595059,"[Equity Point Sea, Spain, 7-8.5, Hs:1, Rt2, na...",[Equity Point Sea]
2,2,3637809,"[The Naughty Squirrel Backpackers Hostel, Latv...",[The Naughty Squirrel Backpackers Hostel]
3,3,3835945,"[Chile, 5-7, Hs:1, Rt3, nationality: England, ...",[]
4,4,3461306,"[Spain, <5, Hs:1, Rt4, nationality: Canada, na...",[]


In [281]:
print user_dist.user_id[2]
print user_dist.user_stuff[2]

3637809
[The Naughty Squirrel Backpackers Hostel, Latvia, >8.5, Hs:1, nationality: Australia, Male, 31-40]


In [283]:
print hostel_dist.hostel[2]
print hostel_dist.hostel_info[2]

The Naughty Squirrel Backpackers Hostel
[>8.5, Awards:16, Latvia, Riga, Linen Included, Free City Maps, Towels Included, Free WiFi, Free Internet Access , Security Lockers, Key Card Access, Common Room, Adaptors, Book Exchange, Hot Showers, Self-Catering Facilities, Hair Dryers, Safe Deposit Box, Steam Room, Internet Access, Laundry Facilities, Luggage Storage, 24 Hour Reception, Tours/Travel Desk, Postal Service, 24 Hour Security, Bar, Tea & Coffee Making Facilities, DVD's, Board Games, Foosball, Credit Cards Accepted, No Curfew, Non Smoking, Taxes Included, H_t6]


In [270]:
# usser distance function 
def j_user_dist(user):
    distance = []
    users1 = []
    hostels = []
    user_a = set(user_dist ['user_stuff'][user_dist.user_id == user].values[0].replace('[','').replace(']','').split(', '))
    
    for i,id in enumerate(user_dist.user_id):
        user_b = set(user_dist.user_stuff[i].replace('[','').replace(']','').split(', '))
        numerator = len(user_a.intersection(user_b)) * 1.0
        denominator = len(user_a.union(user_b)) * 1.0
        distance.append(numerator / denominator)
        users1.append(id)
        hostels.append(user_dist['hsts_liked'][i])
    distances = pd.DataFrame()
    distances['users'] = users1
    distances['distances']=distance
    distances['hostels'] = hostels
    return distances.sort_values('distances', ascending = False)[1:4]

In [271]:
j_user_dist(3298399)

,users,distances,hostels
6563,8144810,0.312500,[Iva Apartments]
3767,9802505,0.300000,"[El Jardin Hostal, Way Hostel, Hostel Rosemary]"
9161,9936860,0.277778,"[Boipeba Bed and Breakfast, Hostal Andalucia]"


In [272]:
for h in list(j_user_dist(1189474)['hostels'].values):
    for H in h.replace('[','').replace(']','').split(', '):
        if H != '':
            print H

In [273]:
# hostel distance function 
def j_hostel_dist(hst,hostel_list):
    distance = []
    hostels1 = []
    links = []
    distances = pd.DataFrame()
    
    hostel_a = set(hostel_dist['hostel_info'][hostel_dist.hostel == hst].values[0].split(', '))
    
    for h in hostel_list:
        
        hostel_b = set(hostel_dist['hostel_info'][hostel_dist['hostel']==h].values[0].split(', '))
        
        numerator = len(hostel_a.intersection(hostel_b)) * 1.0 
        denominator = len(hostel_a.union(hostel_b)) * 1.0
        distance.append(numerator / denominator)
        
        hostels1.append(h)
        links.append(hostel_dist['link'][hostel_dist['hostel']==h].values[0])
        
    distances['hostel'] = hostels1
    distances['distances']=distance
    distances['link'] = links
    
    return distances.sort_values('distances', ascending = False)[:1]

In [274]:
h_list = ['Iva Apartments','El Jardin Hostal','Way Hostel','Hostel Rosemary',
          'Boipeba Bed and Breakfast','Hostal Andalucia','Hvar Out Hostel']

j_hostel_dist('Bed & Bike Barcelona',h_list)

,hostel,distances,link
2,Way Hostel,0.530612,http://www.hostelworld.com/hosteldetails.php/W...


In [275]:
def hostel_rec(user):
    rec = pd.DataFrame()
    
    h_list = []
    for h in list(j_user_dist(user)['hostels'].values):
        for H in h.replace('[','').replace(']','').split(', '):
            if H != '':
                h_list.append(H)
            
    users_hsts = [h.replace('[','').replace(']','')
                 for h in user_dist['hsts_liked'][user_dist['user_id']==user].values[0].split(', ')]
    
    for hst in users_hsts:
        rec = rec.append(j_hostel_dist(hst,h_list),ignore_index=True)
        
    rec['user_hostel'] = users_hsts
    return rec[['user_hostel','hostel','distances','link']]

In [276]:
hostel_rec(1189474)

,user_hostel,hostel,distances,link
0,Fortuna Botel,NaN,NaN,NaN


In [277]:
hostel_rec(1595059)

,user_hostel,hostel,distances,link
0,Equity Point Sea,Equity Point Sea,1.0,http://www.hostelworld.com/hosteldetails.php/E...


In [278]:
hostel_rec(3298399)

,user_hostel,hostel,distances,link
0,Bed & Bike Barcelona,Way Hostel,0.530612,http://www.hostelworld.com/hosteldetails.php/W...
1,B&B Giovy,Hostal Andalucia,0.375000,http://www.hostelworld.com/hosteldetails.php/H...
2,CroParadise Green Hostel,Way Hostel,0.418182,http://www.hostelworld.com/hosteldetails.php/W...
3,B&B Giovy,Hostal Andalucia,0.375000,http://www.hostelworld.com/hosteldetails.php/H...
